### Quantization on MobileNet V2

In [3]:
import torch
print(torch.backends.quantized.supported_engines) #Check for fpgemm

['qnnpack', 'none', 'onednn', 'x86', 'fbgemm']


In [7]:
import torch
from torch import nn
from torchvision.models import mobilenet_v2
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Step 1: Load a pre-trained MobileNetV2 model
model_fp32 = mobilenet_v2(pretrained=True)
model_fp32.eval()

# Step 2: Prepare the data (e.g., ImageNet or a similar dataset)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Using a sample dataset (CIFAR-10 in this case, but replace with your dataset)
dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

# Step 3: Fuse appropriate layers for MobileNetV2
# MobileNetV2 has depthwise separable convolutions, so we only need to fuse the first conv and batch norm layers
model_fp32_fused = model_fp32
for module_name, module in model_fp32_fused.named_children():
    if module_name == "features":
        for idx, m in module.named_children():
            if isinstance(m, nn.Conv2d):
                # Fuse Conv2d + BatchNorm2d + ReLU in each block (if applicable)
                torch.quantization.fuse_modules(model_fp32_fused.features[int(idx)], ['0', '1', '2'], inplace=True)

# Step 4: Specify quantization configuration
# Use fbgemm backend for x86 architectures
model_fp32_fused.qconfig = torch.quantization.get_default_qconfig('fbgemm')

# Step 5: Prepare the model for static quantization
# This inserts observers in the model that will collect statistics
model_int8_prepared = torch.quantization.prepare(model_fp32_fused)

# Step 6: Calibrate the model by running inference
# Here, we pass the data through the model so that it can learn activation ranges
with torch.no_grad():
    for images, _ in dataloader:
        model_int8_prepared(images)

# Step 7: Convert to a quantized model
model_int8 = torch.quantization.convert(model_int8_prepared)

# Step 8: Evaluate the quantized model
def evaluate(model, dataloader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model: {100 * correct / total:.2f}%')

# Evaluate both the original FP32 model and the quantized model
print("Evaluating FP32 Model...")
evaluate(model_fp32, dataloader)

print("Evaluating INT8 Quantized Model...")
evaluate(model_int8, dataloader)


Files already downloaded and verified


/home/sunildj/.local/lib/python3.10/site-packages/torch/ao/quantization/observer.py:221: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


Evaluating FP32 Model...
Accuracy of the model: 0.00%
Evaluating INT8 Quantized Model...


NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [Meta, QuantizedCPU, QuantizedCUDA, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

Meta: registered at ../aten/src/ATen/core/MetaFallback